In [1]:
# imports
from tqdm import tqdm # progressbar
import os
import numpy as np
from astropy.stats import mad_std
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import time


# find SNR peak files
path = "D:/McDonald_analysis/analysis/results"
path_imgs = "D:/McDonald_analysis/analysis/closeups/"
key = "peaks.npy"
for d,s,f in os.walk(path):
    files = [os.path.join(path, file) for file in f if key in file and 'speed' not in file]
files

['D:/McDonald_analysis/analysis/results\\results_M11_peaks.npy',
 'D:/McDonald_analysis/analysis/results\\results_M11_sensitivity_peaks.npy',
 'D:/McDonald_analysis/analysis/results\\results_M11_Sept18_peaks.npy',
 'D:/McDonald_analysis/analysis/results\\results_M11_Sept19_AGtest_peaks.npy',
 'D:/McDonald_analysis/analysis/results\\results_M11_Sept20_2_peaks.npy',
 'D:/McDonald_analysis/analysis/results\\results_M11_Sept20_peaks.npy',
 'D:/McDonald_analysis/analysis/results\\results_M11_Sept21_peaks.npy',
 'D:/McDonald_analysis/analysis/results\\results_M11_Sept22_peaks.npy',
 'D:/McDonald_analysis/analysis/results\\results_M37_peaks.npy',
 'D:/McDonald_analysis/analysis/results\\results_M37_Sept20_peaks.npy',
 'D:/McDonald_analysis/analysis/results\\results_NGC6811_Sept22_peaks.npy',
 'D:/McDonald_analysis/analysis/results\\results_NGC7788_Sept20_peaks.npy',
 'D:/McDonald_analysis/analysis/results\\results_NGC884_peaks.npy',
 'D:/McDonald_analysis/analysis/results\\results_NGC884_sens

In [1]:
# res_local_peaks.append([i, snr_maxs, loc_check, t_ids])
# identify isolated peaks in each file
for file in files:
    
    print(file)
    
    # load the result file
    res = np.load(file, allow_pickle=True)
    
    # all locations
    peak_locs = []
    source_ids = []
    for i in range(len(res)):
        source, peaks, locations, templates = res[i]
        for loc in locations:
            peak_locs.append(loc)
            source_ids.append(source)
            
            
    # find isolated peaks
    window = 1875
    peak_locs = np.array(peak_locs)
    for i,(peak, source) in enumerate(zip(peak_locs, source_ids)):

        # remove the particular peak
        peak_locs_ = np.delete(peak_locs, i)
        diff = np.abs(peak - peak_locs_)

        # is the peak sufficiently isolated?
        if np.all(diff > window) == True:
            #print(source, peak)
            #print('Isolated peak!')

            # Load the .png image file
            subdir = file.split('/')[-1].split('_peaks')[0].split('results\\')[-1]
            img_path = os.path.join(path_imgs, subdir, str(source) + '_' + str(peak) + '.png')
            try:
                img = mpimg.imread(img_path)

                # Plot the image
                plt.figure(figsize=(20,20))
                plt.imshow(img)
                plt.axis('off')  # Optional: Turn off the axes
                plt.show()
            except Exception as e:
                print(e)

NameError: name 'files' is not defined